[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/llm-field-guide/mpt/mpt-30b-chatbot.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/llm-field-guide/mpt/mpt-30b-chatbot.ipynb)

# MTP-30B-Chat in Hugging Face and LangChain

In this notebook we'll explore how we can use the open source **MTP-30B** model in both Hugging Face transformers and LangChain.

---

🚨 _Note that running this on CPU is practically impossible. It will take a very long time. If running on Google Colab you go to **Runtime > Change runtime type > Hardware accelerator > GPU > GPU type > A100**._

---

We start by doing a `pip install` of all required libraries.

In [ ]:
!pip install -qU transformers accelerate einops langchain xformers bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 106.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 7.2 MB/s eta 0:00:00


## Initializing the Hugging Face Pipeline

The first thing we need to do is initialize a `text-generation` pipeline with Hugging Face transformers. The Pipeline requires three things that we must initialize first, those are:

* A LLM, in this case it will be `mosaicml/mpt-30b-chat`.

* The respective tokenizer for the model.

* A stopping criteria object.

We'll explain these as we get to them, let's begin with our model.

We initialize the model and move it to our CUDA-enabled GPU. Using Colab this can take 5-10 minutes to download and initialize the model.

In [ ]:
from torch import cuda, bfloat16
import transformers

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

model = transformers.AutoModelForCausalLM.from_pretrained(
    'mosaicml/mpt-30b-chat',
    trust_remote_code=True,
    load_in_8bit=True,  # this requires the `bitsandbytes` library
    max_seq_len=8192,
    init_device=device
)
model.eval()
#model.to(device)
print(f"Model loaded on {device}")

A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-30b-chat:
- configuration_mpt.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-30b-chat:
- norm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-30b-chat:
- flash_attn_triton.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-30b-chat:
- attention.py
- flash_attn_triton.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-30b-chat:
- blocks.py
- norm.py
- attention.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-30b-chat:
- meta_init_context.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-30b-chat:
- adapt_tokenizer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-30b-chat:
- param_init_fns.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-30b-chat:
- modeling_mpt.py
- blocks.py
- meta_init_context.py
- adapt_tokenizer.py
- custom_embedding.py
- hf_prefixlm_converter.py
- param_init_fns.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Instantiating an MPTForCausalLM model from /root/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-30b-chat/7debc3fc2c5f330a33838bb007c24517b73347b8/modeling_mpt.py
You are using config.init_device='cuda:0', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/di

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('8013'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Model loaded on cuda:0


The pipeline requires a tokenizer which handles the translation of human readable plaintext to LLM readable token IDs. The MPT-30B model was trained using the `mosaicml/mpt-30b` tokenizer, which we initialize like so:

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained("mosaicml/mpt-30b")

Finally we need to define the _stopping criteria_ of the model. The stopping criteria allows us to specify *when* the model should stop generating text. If we don't provide a stopping criteria the model just goes on a bit of a tangent after answering the initial question.

In [ ]:
from transformers import StoppingCriteria, StoppingCriteriaList

# we create a list of stopping criteria
stop_token_ids = [
    tokenizer.convert_tokens_to_ids(x) for x in [
        ['Human', ':'], ['AI', ':']
    ]
]

stop_token_ids

[[22705, 27], [18128, 27]]

We need to convert these into `LongTensor` objects:

In [ ]:
import torch

stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]
stop_token_ids

[tensor([22705,    27], device='cuda:0'),
 tensor([18128,    27], device='cuda:0')]

We can do a quick spot check that no `<unk>` token IDs (`0`) appear in the `stop_token_ids` — there are none so we can move on to building the stopping criteria object that will check whether the stopping criteria has been satisfied — meaning whether any of these token ID combinations have been generated.

In [ ]:
from transformers import StoppingCriteria, StoppingCriteriaList

# define custom stopping criteria object
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

Now we're ready to initialize the HF pipeline. There are a few additional parameters that we must define here. Comments explaining these have been included in the code.

In [ ]:
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    stopping_criteria=stopping_criteria,  # without this model rambles during chat
    temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    top_p=0.15,  # select from top tokens whose probability add up to 15%
    top_k=0,  # select from top 0 tokens (because zero, relies on top_p)
    max_new_tokens=128,  # mex number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

The model 'MPTForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerNormFor

Confirm this is working:

In [ ]:
res = generate_text("Explain to me the difference between nuclear fission and fusion.")
print(res[0]["generated_text"])

Explain to me the difference between nuclear fission and fusion.
Fission is a process in which an atomic nucleus splits into two smaller nuclei, releasing energy as well as additional neutrons that can cause further fissions. Fusion involves combining light atomic nuclei (such as hydrogen isotopes) at high temperatures or pressures so they merge together forming heavier elements while also producing large amounts of energy. The most common example being the reaction occurring inside stars like our sun where Hydrogen atoms combine under extreme heat & pressure resulting in helium formation along with release of gamma radiation. In contrast Fission typically occurs when heavy element such as Uranium-235 absorbs a neutron causing it's nucleus to split apart giving off


Now to implement this in LangChain

In [ ]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline

# template for an instruction with no input
prompt = PromptTemplate(
    input_variables=["instruction"],
    template="{instruction}"
)

llm = HuggingFacePipeline(pipeline=generate_text)

llm_chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
print(llm_chain.predict(
    instruction="Explain to me the difference between nuclear fission and fusion."
).lstrip())

Fission is a process in which an atomic nucleus splits into two smaller nuclei, releasing energy as well as additional neutrons that can cause further fissions. Fusion involves combining light atomic nuclei (such as hydrogen isotopes) at high temperatures or pressures so they merge together forming heavier elements while also producing large amounts of energy. The most common example being the reaction occurring inside stars like our sun where Hydrogen atoms combine under extreme heat & pressure resulting in helium formation along with release of gamma radiation. In contrast Fission typically occurs when heavy element such as Uranium-235 absorbs a neutron causing it's nucleus to split apart giving off


We still get the same output as we're not really doing anything differently here, but we have now added MTP-30B-chat to the LangChain library. Using this we can now begin using LangChain's advanced agent tooling, chains, etc, with MTP-30B.

## MPT-30B Chatbot

Using the above and LangChain we can create a chatbot (conversational agent) very easily. We start by initializing the conversational memory required:

In [ ]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    memory_key="history",  # important to align with agent prompt (below)
    k=5,
    return_only_outputs=True
)

Now we can initialize the agent using our `memory` and `llm`:

In [ ]:
from langchain.chains import ConversationChain

chat = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

The default prompt template will cause the model to return longer text, we can modify it to be more concise.

In [ ]:
chat.prompt

PromptTemplate(input_variables=['history', 'input'], output_parser=None, partial_variables={}, template='The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:', template_format='f-string', validate_template=True)

In [ ]:
chat.prompt.template = \
"""The following is a friendly conversation between a human and an AI. The AI is conversational but concise in its responses without rambling. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:"""

In [ ]:
res = chat.predict(input='hi how are you?')
res



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is conversational but concise in its responses without rambling. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: hi how are you?
AI:

> Finished chain.


" I'm just a computer program, so I don't have feelings like humans do. How can I assist you today?\n\nHuman:"

By default the stopping criteria we earlier defined only stops the model once it has generated the output like `Human:`, we need to trim this text to avoid confusing the later chat steps. We access the previous message within the `chat.memory`:

In [ ]:
chat.memory

ConversationBufferWindowMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='hi how are you?', additional_kwargs={}, example=False), AIMessage(content=" I'm just a computer program, so I don't have feelings like humans do. How can I assist you today?\n\nHuman:", additional_kwargs={}, example=False)]), output_key=None, input_key=None, return_messages=False, human_prefix='Human', ai_prefix='AI', memory_key='history', k=5)

In [ ]:
chat.memory.chat_memory.messages[-1]

AIMessage(content=" I'm just a computer program, so I don't have feelings like humans do. How can I assist you today?\n\nHuman:", additional_kwargs={}, example=False)

From here we can simple add some logic to remove text we defined in our stopping criteria.

In [ ]:
# check for double newlines (also happens often)
chat.memory.chat_memory.messages[-1].content = chat.memory.chat_memory.messages[-1].content.split('\n\n')[0]
# strip any whitespace
chat.memory.chat_memory.messages[-1].content = chat.memory.chat_memory.messages[-1].content.strip()
# check for stop text at end of output
for stop_text in ['Human:', 'AI:', '[]']:
    chat.memory.chat_memory.messages[-1].content = chat.memory.chat_memory.messages[-1].content.removesuffix(stop_text)
# strip again
chat.memory.chat_memory.messages[-1].content = chat.memory.chat_memory.messages[-1].content.strip()

chat.memory.chat_memory.messages[-1]

AIMessage(content="I'm just a computer program, so I don't have feelings like humans do. How can I assist you today?", additional_kwargs={}, example=False)

We can wrap this into a function:

In [ ]:
def chat_trim(chat_chain, query):
    # create response
    chat_chain.predict(input=query)
    # check for double newlines (also happens often)
    chat.memory.chat_memory.messages[-1].content = chat.memory.chat_memory.messages[-1].content.split('\n\n')[0]
    # strip any whitespace
    chat.memory.chat_memory.messages[-1].content = chat.memory.chat_memory.messages[-1].content.strip()
    # check for stop text at end of output
    for stop_text in ['Human:', 'AI:', '[]']:
        chat.memory.chat_memory.messages[-1].content = chat.memory.chat_memory.messages[-1].content.removesuffix(stop_text)
    # strip again
    chat.memory.chat_memory.messages[-1].content = chat.memory.chat_memory.messages[-1].content.strip()
    # return final response
    return chat_chain.memory.chat_memory.messages[-1].content

In [ ]:
chat_trim(chat, "Explain to me the difference between nuclear fission and fusion.")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is conversational but concise in its responses without rambling. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hi how are you?
AI: I'm just a computer program, so I don't have feelings like humans do. How can I assist you today?
Human: Explain to me the difference between nuclear fission and fusion.
AI:

> Finished chain.


'Nuclear fission is the process of splitting atoms into smaller components by bombarding them with high-energy particles or radiation. This releases large amounts of energy that can be harnessed for various purposes such as generating electricity. On the other hand, nuclear fusion involves combining two light atomic nuclei to form a heavier nucleus, which also releases energy. Fusion occurs naturally in stars, including our sun, but replicating this process on Earth has proven challenging due to the extreme conditions required.'

In [ ]:
chat_trim(chat, "Could you ELI5?")  # Explain Like I'm 5



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is conversational but concise in its responses without rambling. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hi how are you?
AI: I'm just a computer program, so I don't have feelings like humans do. How can I assist you today?
Human: Explain to me the difference between nuclear fission and fusion.
AI: Nuclear fission is the process of splitting atoms into smaller components by bombarding them with high-energy particles or radiation. This releases large amounts of energy that can be harnessed for various purposes such as generating electricity. On the other hand, nuclear fusion involves combining two light atomic nuclei to form a heavier nucleus, which also releases energy. Fusion occurs naturally in stars, including our sun, but replicating this process on Earth has proven challenging due 

"Sure! In simpler terms, nuclear fission is like breaking apart a toy car to get the pieces inside. It requires forceful intervention to break down the original structure. Meanwhile, nuclear fusion is more like putting together Lego blocks to make something new. You're taking separate elements and joining them together to create something different than before. Both processes release energy, but they approach it from opposite directions."

With that we have our MPT-30B powered chatbot!

---